# Run pipeline with bash 

You will need to install Salmon, fastQC and multiQC before running the commands below. Otherwise, you can create the conda environment, activate it and run the notebook then, as explained [here](create_conda_env.ipynb). 



### Set pipeline folder

In [ ]:
import os, shutil
pwd = os.getcwd()
pipeline_folder = pwd + "/../" + "rnaseq-nf" 

os.chdir(pipeline_folder)

### Move fastq files and call bash script to rename input files

In [ ]:
fq_folder = pipeline_folder + "/data_chicken"

if not os.path.exists(fq_folder):
    os.makedirs(fq_folder) 
    
!cp data/ggal/ggal_gut*.fq ./data_chicken

os.chdir(fq_folder)

## Calling bash script to rename files
!../bin/rename_files.sh

### Create Salmon index

In [ ]:
os.chdir(pipeline_folder)
!salmon index --threads 1 -t ./data/ggal/ggal_1_48850000_49020000.Ggal71.500bpflank.fa -i index

### Create folders to dump results 

In [ ]:
## Set results folders
r_fastqc = pipeline_folder + "/results_fastqc"
r_salmon = pipeline_folder + "/results_salmon"
d_multiqc = pipeline_folder + "/multiqc_in"
r_multiqc = pipeline_folder + "/results_multiqc"

try:
    shutil.rmtree(r_fastqc)
except OSError:
    pass

try:
    shutil.rmtree(r_salmon)
except OSError:
    pass

try:
    shutil.rmtree(d_multiqc)
except OSError:
    pass

try:
    shutil.rmtree(r_multiqc)
except OSError:
    pass

os.makedirs(r_fastqc)
os.makedirs(d_multiqc)

### Run Salmon and remove index

In [ ]:
!salmon quant --threads 1 --libType=U -i index -1 ./data_chicken/chicken_gut_1.fq -2 ./data_chicken/chicken_gut_2.fq -o results_salmon
!rm -rf index

### Create fastQC results folder and run it

In [ ]:
!fastqc -o results_fastqc -f fastq -q ./data_chicken/*.fq

### Create folders to move input and results for multiQC

In [ ]:
!cp results_fastqc/* multiqc_in
!cp -rf results_salmon/* multiqc_in
!cp multiqc/* multiqc_in
!echo "custom_logo: $PWD/multiqc_in/logo.png" >> multiqc_in/multiqc_config.yaml

os.chdir(d_multiqc)

### Run multiQC

In [ ]:
!multiqc -v . --outdir ../results_multiqc/

### Remove input data for multiQC and calling firefox to display multiQC report 

In [ ]:
os.chdir(r_multiqc)
shutil.rmtree(d_multiqc)

### Uncomment to show the report on firefox, otherwise go to 
### /your_path/BovReg-Reproducibility/rnaseq-nf/results_multiqc and open the file multiqc_report.html
### with your favourite browser

# !firefox multiqc_report.html